In [ ]:
import os
import random
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
from PIL import Image, ImageEnhance
from pathlib import Path

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
DATASET_PATH = Path("COVID-19_Radiography_Dataset/")
BACKUP_PATH = DATASET_PATH.parent / (DATASET_PATH.name + "_backup")
TARGET = 4000  # número objetivo por clase (entre 1345 y 10194)

# =============================
# 1. Conteo inicial
# =============================
classes = [d for d in DATASET_PATH.iterdir() if d.is_dir()]
class_counts = {c.name: len(list(c.glob("*"))) for c in classes}
print("Conteo inicial:", class_counts)

BACKUP_PATH.mkdir(parents=True, exist_ok=True)
random.seed(42)
np.random.seed(42)

# =============================
# 2. Undersampling (para clases con > TARGET)
# =============================
def undersample_class(class_folder: Path, target: int, backup_root: Path):
    files = list(class_folder.glob("*"))
    n = len(files)
    if n <= target:
        return 0
    keep = set(random.sample(files, target))
    moved = 0
    backup_cls = backup_root / class_folder.name
    backup_cls.mkdir(parents=True, exist_ok=True)
    for f in files:
        if f not in keep:
            shutil.move(str(f), str(backup_cls / f.name))
            moved += 1
    return moved

moved_summary = {}
for cls in classes:
    cnt = len(list(cls.glob("*")))
    if cnt > TARGET:
        moved_summary[cls.name] = undersample_class(cls, TARGET, BACKUP_PATH)
print("Imágenes movidas (undersample):", moved_summary)


Conteo inicial: {'COVID': 3616, 'Lung_Opacity': 6012, 'Normal': 10192, 'Viral Pneumonia': 1345}
Imágenes movidas (undersample): {'Lung_Opacity': 2012, 'Normal': 6192}


In [4]:
# Funciones de augmentación
def add_gaussian_noise(pil_img, sigma=8):
    arr = np.array(pil_img).astype(np.float32)
    noise = np.random.normal(0, sigma, arr.shape)
    noisy = np.clip(arr + noise, 0, 255).astype(np.uint8)
    return Image.fromarray(noisy)


def random_affine(pil_img, max_translate=5, max_shear=5):
    w, h = pil_img.size
    trans_x = random.uniform(-max_translate, max_translate)
    trans_y = random.uniform(-max_translate, max_translate)
    shear = random.uniform(-max_shear, max_shear)
    matrix = (1, np.tan(np.radians(shear)), trans_x, np.tan(np.radians(shear)), 1, trans_y)
    return pil_img.transform((w, h), Image.AFFINE, matrix, resample=Image.BILINEAR)

def augment_image(pil_img):
    img = pil_img.copy()
    ops = [
        lambda x: x.transpose(Image.FLIP_LEFT_RIGHT),
        lambda x: x.rotate(random.uniform(-15, 15), resample=Image.BILINEAR),
        lambda x: ImageEnhance.Brightness(x).enhance(random.uniform(0.85, 1.15)),
        lambda x: ImageEnhance.Contrast(x).enhance(random.uniform(0.85, 1.15)),
        lambda x: random_affine(x, max_translate=6, max_shear=3),
        lambda x: add_gaussian_noise(x, sigma=random.uniform(5,12)),
    ]
    for op in random.sample(ops, random.choice([2,3])):
        try: img = op(img)
        except: pass
    return img

FINAL_SIZE = (100,100)

def generate_until_target(class_folder: Path, target: int):
    current_files = list(class_folder.glob("*"))
    count = len(current_files)
    created, idx = 0, 0
    originals = [f for f in current_files if "_aug" not in f.name]
    pbar = tqdm(total=max(0, target - count), desc=f"Aug {class_folder.name}")
    while count < target:
        src = random.choice(originals)
        try:
            img = Image.open(src).convert("L")
        except: continue
        aug = augment_image(img).resize(FINAL_SIZE, Image.BILINEAR)
        new_name = f"{src.stem}_aug{idx:04d}{src.suffix}"
        aug.save(class_folder / new_name)
        idx += 1
        created += 1
        count += 1
        pbar.update(1)
    pbar.close()
    return created

created_summary = {}
for cls in classes:
    cnt = len(list(cls.glob("*")))
    if cnt < TARGET:
        created_summary[cls.name] = generate_until_target(cls, TARGET)
print("Imágenes creadas (augmentation):", created_summary)

final_counts = {c.name: len(list(c.glob('*'))) for c in classes}
print("Conteo final por clase:", final_counts)


Aug Viral Pneumonia: 100%|██████████| 2655/2655 [00:31<00:00, 85.16it/s] 


Imágenes creadas (augmentation): {'COVID': 384, 'Viral Pneumonia': 2655}
Conteo final por clase: {'COVID': 4000, 'Lung_Opacity': 4000, 'Normal': 4000, 'Viral Pneumonia': 4000}


In [5]:
# Crear DataFrame actualizado
records = []
for cls in classes:
    for f in cls.glob("*"):
        records.append((str(f), cls.name))
df = pd.DataFrame(records, columns=["image_path", "label"])

label_encoder = LabelEncoder()
df["label_encoded"] = label_encoder.fit_transform(df["label"])
print("Total de imágenes balanceadas:", len(df))
print(df["label"].value_counts())

# División en conjuntos
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df["label_encoded"], random_state=42)
val_df, test_df = train_test_split(test_df, test_size=0.5, stratify=test_df["label_encoded"], random_state=42)

# Transformaciones PyTorch
train_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Dataset personalizado
class ChestXrayDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.data = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx]["image_path"]
        label = self.data.iloc[idx]["label_encoded"]
        image = Image.open(img_path).convert("L").resize((100,100))
        if self.transform:
            image = self.transform(image)
        return image, label

train_loader = DataLoader(ChestXrayDataset(train_df, train_transforms), batch_size=32, shuffle=True, num_workers=2)
val_loader   = DataLoader(ChestXrayDataset(val_df, test_transforms), batch_size=32, shuffle=False, num_workers=2)
test_loader  = DataLoader(ChestXrayDataset(test_df, test_transforms), batch_size=32, shuffle=False, num_workers=2)


Total de imágenes balanceadas: 16000
label
COVID              4000
Lung_Opacity       4000
Normal             4000
Viral Pneumonia    4000
Name: count, dtype: int64


## PERCEPTRON MULTICAPA MLP


In [6]:
# =====================================
# MÓDULO 3.1: MLP (Perceptrón Multicapa)
# =====================================

import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Entrenando en:", device)


Entrenando en: cpu


### Modelo base con torch 
Para probar el preprocesamiento 

recibirá una imagen (1 canal, 100x100 píxeles) → la aplana → pasa por varias capas densas con Dropout

In [7]:
class MLP_Base(nn.Module):
    def __init__(self, input_size=100*100, num_classes=4, dropout_rate=0.3):
        super(MLP_Base, self).__init__()
        
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_size, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 128)
        self.fc4 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(dropout_rate)
        
    def forward(self, x):
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)
        return x

# Crear modelo
mlp_model = MLP_Base().to(device)
print(mlp_model)


MLP_Base(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=10000, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=4, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)


In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mlp_model.parameters(), lr=0.001, weight_decay=1e-4)  # regularización L2


In [9]:
def train_mlp(model, train_loader, val_loader, criterion, optimizer, epochs=10):
    train_losses, val_losses, val_accuracies = [], [], []

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
        
        epoch_loss = running_loss / len(train_loader.dataset)
        train_losses.append(epoch_loss)

        # Validación
        model.eval()
        val_loss, correct, total = 0.0, 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)
                
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_loss /= len(val_loader.dataset)
        val_acc = 100 * correct / total
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        print(f"Época [{epoch+1}/{epochs}] - "
              f"Pérdida entrenamiento: {epoch_loss:.4f} | "
              f"Pérdida validación: {val_loss:.4f} | "
              f"Accuracy validación: {val_acc:.2f}%")
    
    return train_losses, val_losses, val_accuracies


In [ ]:
train_losses, val_losses, val_acc = train_mlp(
    mlp_model, train_loader, val_loader, criterion, optimizer, epochs=10
)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

mlp_model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = mlp_model(images)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

print("Reporte de clasificación (Test):")
print(classification_report(y_true, y_pred, target_names=label_encoder.classes_))


In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(train_losses, label="Entrenamiento")
plt.plot(val_losses, label="Validación")
plt.title("Evolución de la pérdida")
plt.legend()

plt.subplot(1,2,2)
plt.plot(val_acc, label="Accuracy Validación")
plt.title("Evolución del accuracy")
plt.legend()
plt.show()
